<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/table1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stat
!pip install fexact
import fexact  # https://github.com/boussoffara/fexact

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 1.9 MB/s eta 0:00:00


In [4]:
# read data
df = pd.read_csv('https://api.vitaldb.net/cases')
print(df.columns)
df

Index(['caseid', 'subjectid', 'casestart', 'caseend', 'anestart', 'aneend',
       'opstart', 'opend', 'adm', 'dis', 'icu_days', 'death_inhosp', 'age',
       'sex', 'height', 'weight', 'bmi', 'asa', 'emop', 'department', 'optype',
       'dx', 'opname', 'approach', 'position', 'ane_type', 'preop_htn',
       'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb', 'preop_plt',
       'preop_pt', 'preop_aptt', 'preop_na', 'preop_k', 'preop_gluc',
       'preop_alb', 'preop_ast', 'preop_alt', 'preop_bun', 'preop_cr',
       'preop_ph', 'preop_hco3', 'preop_be', 'preop_pao2', 'preop_paco2',
       'preop_sao2', 'cormack', 'airway', 'tubesize', 'dltubesize', 'lmasize',
       'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'intraop_ebl',
       'intraop_uo', 'intraop_rbc', 'intraop_ffp', 'intraop_crystalloid',
       'intraop_colloid', 'intraop_ppf', 'intraop_mdz', 'intraop_ftn',
       'intraop_rocu', 'intraop_vecu', 'intraop_eph', 'intraop_phe',
       'intraop_epi', 'intraop_ca'],
     

,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,...,0,120,0.0,100,70,0,10,0,0,0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,...,0,150,0.0,0,100,0,20,0,0,0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,...,0,0,0.0,0,50,0,0,0,0,0
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,...,0,80,0.0,100,100,0,50,0,0,0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,...,0,0,0.0,0,160,0,10,900,0,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,6384,5583,0,15248,-260,15640.0,2140,14140,-215340,648660,...,0,150,0.0,0,90,0,20,0,0,0
6384,6385,2278,0,20643,-544,20996.0,2396,19496,-225600,1675200,...,0,100,0.0,0,100,0,25,30,0,300
6385,6386,4045,0,19451,-667,19133.0,3533,18233,-200460,836340,...,0,70,0.0,0,130,0,10,0,0,0
6386,6387,5230,0,12025,-550,12830.0,1730,11030,-227760,377040,...,0,120,0.0,0,50,0,0,0,0,0


In [5]:
# add columns
df['opdur'] = df['opend'] - df['opstart']
df['anedur'] = df['aneend'] - df['anestart']
df['hospdur'] = df['dis'] - df['adm']

# remove columns
df.drop(columns=['opstart', 'opend', 'anestart', 'aneend', 'dis', 'adm'], inplace=True)
df = df.loc[:, ~df.columns.str.endswith('id')]

# group variable
grpvar = 'death_inhosp'

# manually set categorical variables
catvars = ['department']

In [7]:
def format_pval(pval):
    '''Returns the formated string of a p-value'''
    if pval < 0.001:
        return '< 0.001'
    return f'{pval:.3f}'

def format_number(num, prec=3):
    '''Returns the formated number up to specific precision'''
    fmt = '{:,.' + str(prec) + 'f}'
    s = fmt.format(num)
    return s.rstrip('0').rstrip('.')

def fisher_exact(table):
    '''Returns p-value for the Fisher's exact test of nxm contingency table
    fisher_exact([[8,2,12], [1,5,2]])  # 0.011825369366598752
    '''
    return fexact.fexact(np.array(table))

print(format_pval(1e-5))
print(format_number(6388))

< 0.001
6,388


In [9]:
import warnings

# convert boolean type to int --> remains are float or str
df.replace({False: 0, True: 1}, inplace=True)

if grpvar is not None:
    grp_names = np.unique(df[grpvar])

# for csv file
rows = []

# Generate table header
tabs = ['', 'Total']
if grpvar is not None:
    for grp_name in grp_names:
        tabs.append(f'{grpvar}={grp_name}')
    if len(grp_names) > 1:
        tabs.append('P-value')
        tabs.append('Test')
rows.append(tabs)

tabs = ['n']
if grpvar is not None:
    tabs.append(format_number(sum(~df[grpvar].isnull())))
    for grp_name in grp_names:
        tabs.append(format_number(sum(df[grpvar] == grp_name)) + ' (' + format_number(np.mean(df[grpvar] == grp_name) * 100, 1) + '%)')
else:
    tabs.append(format_number(len(df)))
rows.append(tabs)

# Generate statistics for each variable
for col in df.columns:
    if col == grpvar:
        continue
    try:
        pd.to_numeric(df[col])
        isstr = False
    except:
        isstr = True

    unique_values = sorted(df.loc[~df[col].isnull(), col].unique())  # unique values

    iscat = len(unique_values) < 8
    if col in catvars:
        iscat = True

    if isstr and not iscat:
        continue

    if iscat:  # categorical variables --> represents as count (percent)
        if grpvar is not None:  # create cross table (value x grp)
            xtab = pd.crosstab(df[col], df[grpvar]).fillna(0)
            pval = None
            if len(grp_names) > 1: # NEJM requires Exact method for all categorical variables
                if (xtab > 5).all(axis=None):  # if there is an incidence < 5
                    pval = stat.chi2_contingency(xtab)[1]
                    test_name = 'Chi-square'
                else:
                    pval = fisher_exact(xtab.T.values)
                    test_name = 'Fisher\'s exact'

        is_binary = (len(unique_values) == 2) and (unique_values[0] == 0 and unique_values[1] == 1)
        if is_binary:  # binary
            # print total
            tabs = [col, format_number(sum(df[col] == 1)) + ' (' + format_number(np.mean(df[col] == 1) * 100, 1) + '%)']
            if grpvar is not None: # print group values
                for grp_name in grp_names:
                    grp_mask = (df[grpvar] == grp_name)
                    tabs.append(format_number(sum(df.loc[grp_mask, col])) + ' (' + format_number(np.mean(df.loc[grp_mask, col])*100, 1) + '%)')
                if pval is not None:
                    tabs.append(format_pval(pval))
                    tabs.append(test_name)
            rows.append(tabs)
        else:
            for uval in unique_values:
                # print total
                tabs = [f'{col}={uval}', format_number(sum(df[col] == uval)) + ' (' + format_number(np.mean(df[col] == uval) * 100, 1) + '%)']
                if grpvar is not None: # print group values
                    for grp_name in grp_names:
                        grp_mask = (df[grpvar] == grp_name)
                        tabs.append(format_number(sum(df.loc[grp_mask, col] == uval)) + ' (' + format_number(np.mean(df.loc[grp_mask, col] == uval) * 100, 1) + '%)')
                    if pval is not None:
                        if uval == unique_values[0]:
                            tabs.append(format_pval(pval))
                            tabs.append(test_name)
                rows.append(tabs)

    else:  # continuous variables --> represents as mean (SD)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            isnorm = stat.shapiro(df[col])[1] > 0.05  # check if it is normal distribution
        if isnorm:  # normal distribution
            # print total
            m = df[col].mean()
            s = df[col].std()
            tabs = [col, f'{m:.3f} ({s:.3f})']

            if grpvar is not None:
                # extract group values
                grp_vals = []
                for grp_name in grp_names:
                    a = df.loc[df[grpvar] == grp_name, col]
                    grp_vals.append(a[~a.isnull()])

                # print group values
                for igrp in range(len(grp_vals)):
                    m = grp_vals[igrp].mean()
                    s = grp_vals[igrp].std()
                    tabs.append(f'{m:.3f} ({s:.3f})')

                # print stats
                if len(grp_names) == 2:
                    equal_var = stat.levene(grp_vals[0], grp_vals[1])[1] > 0.05  # levene
                    pval = stat.ttest_ind(grp_vals[0], grp_vals[1], equal_var=equal_var)[1]
                    test_name = 'T-test'
                else:  # 3 or more groups -> anova
                    equal_var = stat.levene(*grp_vals)[1] > 0.05  # levene + homoscedasticity
                    if equal_var:
                        pval = stat.f_oneway(*grp_vals)[1]
                        test_name = 'One-way ANOVA'
                    else:
                        pval = stat.kruskal(*grp_vals)[1]
                        test_name = 'Kruskal-Wallis'
                    tabs.append(format_pval(pval))
                    tabs.append(test_name)
        else:  # non-normal
            # print total
            m = df[col].median()
            q1 = df[col].quantile(0.25)
            q2 = df[col].quantile(0.75)
            tabs = [col, format_number(m, 3) + ' (' + format_number(q1, 3) + '-' + format_number(q2, 3) + ')']

            if grpvar is not None:
                # extract group values
                grp_vals = []
                for grp_name in grp_names:
                    a = df.loc[df[grpvar] == grp_name, col]
                    grp_vals.append(a[~a.isnull()])

                # print group value
                for igrp in range(len(grp_vals)):
                    m = grp_vals[igrp].mean()
                    q1 = grp_vals[igrp].quantile(0.25)
                    q2 = grp_vals[igrp].quantile(0.75)
                    tabs.append(format_number(m, 3) + ' (' + format_number(q1, 3) + '-' + format_number(q2, 3) + ')')

                # print stats
                if len(grp_vals) == 2:
                    pval = stat.mannwhitneyu(grp_vals[0], grp_vals[1], alternative='two-sided')[1]
                    test_name = 'Mann-Whitney'
                elif len(grp_vals) > 2:  # > 3 groups
                    pval = stat.kruskal(*grp_vals)[1]
                    test_name = 'Kruskal-Wallis'
                tabs.append(format_pval(pval))
                tabs.append(test_name)

        rows.append(tabs)

df_results = pd.DataFrame(rows)
df_results.to_csv('table1.csv', index=False, header=False)
df_results

<ipython-input-9-ea9d062e6508>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace({False: 0, True: 1}, inplace=True)


,0,1,2,3,4,5
0,,Total,death_inhosp=0,death_inhosp=1,P-value,Test
1,n,"6,388","6,331 (99.1%)",57 (0.9%),None,None
2,casestart=0,"6,388 (100%)","6,331 (100%)",57 (100%),1.000,Chi-square
3,caseend,"9,924.5 (6,194.5-15,072.75)","11,342.646 (6,189-15,053)","12,019 (6,655-16,200)",0.380,Mann-Whitney
4,icu_days,0 (0-0),0.456 (0-0),11.281 (0-16),< 0.001,Mann-Whitney
...,...,...,...,...,...,...
74,intraop_epi,0 (0-0),1.979 (0-0),664.93 (0-0),< 0.001,Mann-Whitney
75,intraop_ca,0 (0-0),115.998 (0-0),652.105 (0-300),< 0.001,Mann-Whitney
76,opdur,"6,600 (3,600-11,400)","8,153.681 (3,600-11,400)","8,764.14 (3,900-12,243)",0.423,Mann-Whitney
77,anedur,"10,500 (6,720-15,600)","-571,109.455 (6,690-15,600)","12,844.211 (7,140-16,500)",0.311,Mann-Whitney
